In [35]:
import json, newspaper
import feedparser as fp
from newspaper import Article
from time import mktime
from datetime import datetime
import pandas as pd

In [10]:
with open("news_sites.json",'r') as file:
    companies = json.load(file)

In [11]:
companies

{'cnn': {'link': 'http://edition.cnn.com/'},
 'bbc': {'link': 'http://www.bbc.com/'},
 'theguardian': {'link': 'https://www.theguardian.com/international'},
 'breitbart': {'link': 'http://www.breitbart.com/'},
 'infowars': {'link': 'https://www.infowars.com/'},
 'foxnews': {'link': 'http://www.foxnews.com/'},
 'nbcnews': {'link': 'http://www.nbcnews.com/'},
 'washingtonpost': {'link': 'https://www.washingtonpost.com/'},
 'theonion': {'link': 'http://www.theonion.com/'}}

In [25]:
# Set the limit for number of articles to download
LIMIT = 10000

data = {}
data['newspapers'] = {}

In [30]:
count = 1
articles = []
# Iterate through each news company
for company, value in companies.items():
    print("Building site for ", company)
    paper = newspaper.build(value['link'],language='en',memoize_articles=False)
    noneTypeCount = 0
    for content in paper.articles:
        if count > LIMIT:
            break
        try:
            content.download()
            content.parse()
        except Exception as e:
            print(e)
            print("continuing...")
            continue
        # Again, for consistency, if there is no found publish date the article will be skipped.
        # After 10 downloaded articles from the same newspaper without publish date, the company will be skipped.
        if content.publish_date is None:
            print(count, "Article has date of type None...")
            noneTypeCount = noneTypeCount + 1
            if noneTypeCount > 10:
                print("Too many noneType dates, aborting...")
                noneTypeCount = 0
                break
            count = count + 1
            continue
        article = {}
        article['title'] = content.title
        article['text'] = content.text
        article['link'] = content.url
        article['published'] = content.publish_date.isoformat()
        articles.append(article)
        print(count, "article downloaded from", company, "using newspaper, url: ", content.url)
        count = count + 1
        noneTypeCount = 0

Building site for  cnn
1 Article has date of type None...
2 Article has date of type None...
3 Article has date of type None...
4 Article has date of type None...
Article `download()` failed with 404 Client Error: Not Found for url: http://edition.cnn.com/date/2022-11-28 on URL http://edition.cnn.com/date/2022-11-28
continuing...
Article `download()` failed with 404 Client Error: Not Found for url: http://edition.cnn.com/date/2022-11-29 on URL http://edition.cnn.com/date/2022-11-29
continuing...
Article `download()` failed with 404 Client Error: Not Found for url: http://edition.cnn.com/date/2022-11-30 on URL http://edition.cnn.com/date/2022-11-30
continuing...
Article `download()` failed with 404 Client Error: Not Found for url: http://edition.cnn.com/date/2022-12-01 on URL http://edition.cnn.com/date/2022-12-01
continuing...
Article `download()` failed with 404 Client Error: Not Found for url: http://edition.cnn.com/date/2022-12-02 on URL http://edition.cnn.com/date/2022-12-02
contin

KeyboardInterrupt: 

In [37]:
articles = pd.DataFrame(articles)
articles.head()

,title,text,link,published
0,Ukrainian President Volodymyr Zelensky is name...,CNN —\n\nTIME magazine has named Volodymyr Zel...,http://edition.cnn.com/2022/12/07/europe/zelen...,2022-12-07T00:00:00
1,Xi Jinping: Chinese president lands in Saudi A...,CNN —\n\nChinese President Xi Jinping landed i...,http://edition.cnn.com/2022/12/07/middleeast/c...,2022-12-07T00:00:00
2,Germany arrests 25 suspected far-right extremi...,CNN —\n\nGerman officials arrested 22 suspecte...,http://edition.cnn.com/2022/12/07/europe/germa...,2022-12-07T00:00:00
3,Xi Jinping: Saudi's MBS rolls out the red carp...,Editor’s Note: A version of this story appears...,http://edition.cnn.com/2022/12/07/middleeast/x...,2022-12-07T00:00:00
4,Jamaica: President Andrew Holness declares sta...,CNN —\n\nA widespread state of emergency has b...,http://edition.cnn.com/2022/12/07/americas/jam...,2022-12-07T00:00:00


In [ ]:
articles.to_csv('news_articles.csv')